In all of the problems below, the subject of study is the answer data (8 .csv files) for each class you "created" (in the Kahoot environment), which you can import by running this command:

In [ ]:
!wget https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/midterm1.zip
!unzip midterm1.zip

--2023-04-03 14:46:21--  https://raw.githubusercontent.com/NshanPotikyan/Dasa1Doom/master/files/midterm1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22873 (22K) [application/zip]
Saving to: ‘midterm1.zip.1’

midterm1.zip.1      100%[===================>]  22.34K  --.-KB/s    in 0.001s  

2023-04-03 14:46:22 (29.6 MB/s) - ‘midterm1.zip.1’ saved [22873/22873]

Archive:  midterm1.zip
replace gender.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Each kahoot{i}.csv file contains the following data about the ith kahoot:

* question_id - is the identifier of the question
* student_id - is the student identifier
* correct - contains a value of 0 or 1 and indicates whether the given student answered the question correctly (correct=1) or not (correct=0)
* time - shows the time spent on the question in seconds

| question_id |	student_id |	correct	| time |
| --- | --- | --- | --- |
| 1 |	27|	1	| 9.488 |
| 1 |	28|	1	| 4.795 |
| 1 |	25|	1	| 3.873 |
| 1 |	24|	1	| 10.771 |
| 1 |	23|	1	| 4.418 |
| 1 |	22|	1	| 5.688 |
| 1 |	32|	1	| 3.935 |
| 1 |	1	| 1 |	3.929 |
| 1 |	35 |	0	| 30.0 |
| ... |

Each kahoot{i}_questions.csv file contains the following data about the ith kahoot:

* question_id - is the identifier of the question
* question - is the wording of the question
* answer1,2,3,4 - columns contain 4 answer options
* total_time - shows the time allocated to the question in seconds

A ``gender.csv`` file is also provided, which contains the gender data of each student.

In [ ]:
import numpy as np
import pandas as pd

**Problem 1.⭐** It is required to complete the ``function1`` function so that it takes the kahoots data of all classes (kahoot{i}.csv) and combines them to create one table, where the data will be in the following format:

| kahoot_id | question_id |	student_id |	correct	| time |
| --- | --- | --- | --- | --- |
| 1 | 1 |	27|	1	| 9.488 |
| 1 | 1 |	28|	1	| 4.795 |
| 1 | 1 |	25|	1	| 3.873 |
|  ...  |
| 8 | 1 |	1	| 1 |	3.929 |
| 8 | 1 |	35 |	0	| 30.0 |

The function should

* return the resulting table as a pandas.DataFrame
* work similarly on kahoot{i}_questions.csv files.


In [ ]:
def function1(file_names):
  dfs = []
  for file_name in file_names:
    df = pd.read_csv(file_name)
    dfs.append(df)
  final_df = pd.concat(dfs)
  final_df.insert(0, 'kahoot_id', pd.concat([pd.Series([idx]*len(df)) for idx, df in enumerate(dfs, start=1)]))
  final_df.reset_index(drop=True, inplace=True)
  return final_df

In [ ]:
students_data = function1(file_names=[f'kahoot{idx}.csv' for idx in range(1, 9)])
questions_data = function1(file_names=[f'kahoot{idx}_questions.csv' for idx in range(1, 9)])
print(students_data.head())
print(students_data.tail())
print(questions_data.head())
print(questions_data.tail())

   kahoot_id  question_id  student_id  correct    time
0          1            1          27        1   9.488
1          1            1          28        1   4.795
2          1            1          25        1   3.873
3          1            1          24        1  10.771
4          1            1          23        1   4.418
      kahoot_id  question_id  student_id  correct    time
2075          8           10          17        1  23.136
2076          8           10          26        0  38.939
2077          8           10          31        1  24.857
2078          8           10           7        0  48.455
2079          8           10           2        0  16.124
   kahoot_id  question_id                                           question  \
0          1            1  Նշվածներից ո՞րն է տվյալների պատկերման գրադարան...   
1          1            2  Նշվածներից ո՞րն է տվյալների մաքրման գործընթացի...   
2          1            3                    Ի՞նչ է այս գծապատկերի անվանումը   
3 

**Problem 2.⭐** What is required is to complete the ``function2`` function so that it takes the data obtained in **Problem 1** and returns pandas.Series, where
each value represents the identifier of a student who
* participated in all classes' quizzes and on average answered the questions the fastest (``fast answers``)

* participated in only two kahuti (``rare visitor``)

* gave the most correct answers on average (``most correct answers``)

* is the slowest to answer questions on average ("slow answers")

The final row should look like this, where the corresponding student IDs should be in place of the x's:

| | |
| --- | --- |
| fast answers | x |
| rare visitor | x |
| most correct answers | x |
| slow answers | x |

In [ ]:
def function2(data):
  result_dict = {}
  grouped_df = data.groupby(by=['kahoot_id', 'student_id'])
  mean_times_spend = grouped_df.apply(lambda x: x[x["correct"] == 1]["time"].mean())
  students = mean_times_spend.reset_index()['student_id'].value_counts()
  kahoot_counts = len(data["kahoot_id"].unique())
  students_participated_all = students[students == kahoot_counts].index.tolist()
  filtered_df = data[data["student_id"].isin(students_participated_all)]
  filtered_grouped = filtered_df.groupby("student_id")
  mean_filtered = filtered_grouped.apply(lambda x: x[x["correct"] == 1]["time"].mean())
  fastest_student = mean_filtered.idxmin()
  result_dict['fast answers'] = fastest_student
  result_dict['rare visitor'] = students[students == 2].index.tolist()[0]
  mean_correct = data.groupby("student_id")["correct"].mean()
  result_dict['most correct answers'] = mean_correct[mean_correct == mean_correct.max()].index.tolist()[0]
  mean_time = data.groupby("student_id")["time"].mean()
  result_dict['slow answers'] = mean_time[mean_time == mean_time.max()].index.tolist()[0]
  return pd.Series(result_dict)

In [ ]:
print(function2(students_data))

fast answers             2
rare visitor            13
most correct answers    27
slow answers            21
dtype: int64


**Problem 3.⭐⭐** Modify the ``function3`` function so that it takes the data obtained in **Problem 1** and returns a table where the indexes are the student IDs and the columns are the proportions of correct answers to each box. For example, if a student answered 2 out of 10 questions correctly in the first class, then the corresponding value will be 0.2, and if he did not participate, then 0 (he had 0 correct answers) and so on.

The table should also include the data of students who did not participate in any class, given that there are 35 students and the identifiers are sequential numbers starting from 1.

The final table (pandas.DataFrame) should look like this:

| | lab_1 | lab_2 | lab_3 | ... | lab_8 |
| --- | --- | --- | --- | --- | --- |
| 1 | x | x | x | x | x |
| 2 | x | x | x | x | x |
| 3 | x | x | x | x | x |
| ... | x | x | x | x | x |
| 34 | x | x | x | x | x |
| 35 | x | x | x | x | x |

In [ ]:
def function3(data):
  df = data.groupby(['student_id', 'kahoot_id'])['correct'].mean().unstack()
  df.columns = [f'lab_{i}' for i in df.columns]
  df = df.rename_axis(None, axis=0).fillna(0)
  df = df.reindex(range(1, 36), fill_value=0)
  return df

In [ ]:
absences_data = function3(students_data)
print(absences_data)

    lab_1  lab_2  lab_3  lab_4  lab_5  lab_6  lab_7  lab_8
1     1.0    0.8    0.5    0.8    0.5    0.6    0.7    0.3
2     0.7    1.0    0.8    0.9    0.2    0.9    0.8    0.5
3     0.0    0.0    0.6    0.6    0.0    0.8    0.9    0.6
4     0.0    0.0    0.0    0.3    0.6    0.3    0.0    0.3
5     0.9    0.8    0.4    0.7    0.9    0.7    0.7    0.3
6     0.8    0.0    0.0    0.7    0.5    0.2    0.9    0.6
7     0.9    0.8    0.8    0.7    0.6    0.8    0.0    0.5
8     0.8    0.8    0.7    1.0    0.7    0.8    0.0    0.5
9     0.0    1.0    0.8    0.8    0.3    0.9    0.0    0.0
10    0.0    0.5    0.0    0.7    0.4    0.2    0.0    0.5
11    0.8    0.0    0.0    0.3    0.5    0.5    0.4    0.4
12    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
13    0.0    0.0    0.0    0.0    0.7    0.9    0.0    0.0
14    0.0    0.7    0.7    0.8    0.6    0.3    0.6    0.6
15    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
16    0.7    0.6    0.5    0.7    0.5    0.7    0.6    0

**Problem 4.⭐⭐** By adding the data in ``gender.csv`` to the data from the previous problem, find out who answers more questions on average: male or female? To find out the following, you need to fill in the function ``function4`` so that it takes the data obtained in Problem 3 and returns
* ``pandas.Series``, whose indexes will be the gender, and the values will be the averaged proportions of correct answers by gender.

|    |    |
| --- | --- |
| female | x |
| male   | x |

* the data related to the gender of the students, as ``pandas.Series``, where the indices will be the identifiers of the students.

|    |    |
| --- | --- |
| 1 | male |
| 2   | female |
| ... | ... |
| 34 | male |
| 35   | female |


In [ ]:
def function4(data, gender_csv):
  gender_df = pd.read_csv(gender_csv, header=None)
  gender_df = gender_df.set_index(0).rename_axis(None, axis=0)
  gender_df.columns = ['gender']
  merged_df = pd.merge(data, gender_df, left_index=True, right_index=True)
  avg_gender = pd.Series(merged_df.groupby('gender').mean().mean(axis=1).to_dict())
  return avg_gender, gender_df

In [ ]:
result, gender_data = function4(absences_data, gender_csv='gender.csv')
print(result)
print(gender_data)

female    0.515909
male      0.480769
dtype: float64
    gender
1   female
2     male
3   female
4     male
5   female
6   female
7   female
8   female
9   female
10    male
11  female
12  female
13  female
14  female
15  female
16  female
17    male
18    male
19  female
20    male
21    male
22  female
23  female
24    male
25  female
26  female
27    male
28    male
29  female
30  female
31    male
32    male
33    male
34  female
35  female


**Problem 5.⭐⭐⭐** Complete function5 so that it
1. take all initial data (students_data, questions_data, gender_data),
2. join them to students_data with the corresponding columns (student_id, question_id);
3. create a new column named **avg_time**, which should contain the average "thinking" time of the given student based on the previous questions, setting the first value to 0; for example, if he spent 20 and 30 seconds on the first 2 questions, then the first 3 data in the **avg_time** column for the *given student* will be 0, 20, 25.
4. create a new column named **avg_correct**, which should contain the average proportion of correct answers given by the given student based on the previous questions, setting the first value to 1.5. for example, if the values of the correct column of the first 3 questions were 1, 1, 0, then the first 4 data of the **avg_correct** column for the given student will be 1, 1, 1, 0.66.

5. save the following columns in the specified order **gender, avg_time, avg_correct, total_time, question, answer1, answer2, answer3, answer4, correct**
6. divide the data into 2 parts: in the first part 1-5 (train), in the second part 6-8 questions (test)
7. return train, test tables as pandas.DataFrames in original row order

In [ ]:
def function5(students_data, questions_data, gender_data):
  merged_df = pd.merge(students_data, questions_data, on=['kahoot_id', 'question_id'])
  merged_df = pd.merge(merged_df, gender_data, left_on='student_id', right_index=True)
  grouped_merged_df = merged_df.groupby(['student_id', 'kahoot_id'])
  merged_df['avg_time'] = grouped_merged_df['time'].expanding().mean().reset_index(drop=True)
  merged_df['avg_time'] = merged_df['avg_time'].mask(grouped_merged_df['avg_time'].shift().isnull(), 0)
  merged_df['avg_correct'] = grouped_merged_df['correct'].expanding().mean().reset_index(drop=True)
  merged_df['avg_correct'] = merged_df['avg_correct'].mask(grouped_merged_df['avg_correct'].shift().isnull(), 1)
  remaining_columns = ['gender', 'avg_time', 'avg_correct', 'total_time', 'question', 'answer1', 'answer2', 'answer3', 'answer4', 'correct']
  train = merged_df[merged_df['kahoot_id'].between(1, 5)][remaining_columns]
  test = merged_df[merged_df['kahoot_id'].between(6, 9)][remaining_columns]
  return train, test

In [ ]:
train, test = function5(students_data, questions_data, gender_data)
print(train.shape, test.shape)

(1290, 10) (790, 10)


**Problem 6.⭐⭐⭐** In fact, we want to build such a classifier, which will give an opportunity for each student to estimate the probability of him answering a certain question correctly, taking into account the data of that student and the complexity of the question.

Student data will be used by him
* gender,
* average time to answer previous questions (avg_time),
* average proportion of correct answers to previous questions (avg_correct)

As a description of the complexity of the question, we will use

* time allocated to the question (total_time)
* the number of words in the content of the question (question)
* the sum of the number of words in the 4 options of the answer to the question (answer1,2,3,4)

We will use the correct column data as a label.

We need to complete function6 so that it constructs sklearn.pipeline.Pipeline in the following steps:
* sklearn.preprocessing.OrdinalEncoder applied to the gender column
* sklearn.preprocessing.FunctionTransformer applied to the question column with a function that will return the number of words ignoring characters other than letters [[hint]](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)
* sklearn.preprocessing.FunctionTransformer applied to columns answer1,2,3,4 with a function that will return the sum of the number of words in the 4 options (count as in the previous point)
* then apply sklearn.preprocessing.StandardScaler to all columns of the input data (X).
* and use sklearn.neighbors:KNeighborsClassifier with k=7 as classifier

You can use [this] (https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn-compose-columntransformer) to modify individual columns. It is required to return the trained pipeline object on the train set and the accuracy of the predictions on the test set (you can use the score method).



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer


def function6(train, test):
  y_train = train['correct']
  X_train = train.drop(['correct'], axis=1)
  y_test = test['correct']
  X_test = test.drop(['correct'], axis=1)

  col_1 = ColumnTransformer([('gender', OrdinalEncoder(categories=[['male', 'female']]), ['gender'])])
  col_2 = ColumnTransformer([('question', CountVectorizer(token_pattern=r'\b\w+\b', lowercase=True), ['question'])])
  col_3 = ColumnTransformer([('anserws', CountVectorizer(token_pattern=r'\b\w+\b', lowercase=True), ['answer1', 'answer2', 'answer3', 'answer4'])])
  pipeline = Pipeline([
        ('encode_gender', col_1),
        ('countvec_question', col_2),
        ('countvec_answers', col_3),
        ('scale', StandardScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=7))
    ])
  pipeline.fit(X_train, y_train)

    # Evaluate the pipeline on the test data
  score = pipeline.score(X_test, y_test)
  return score

In [ ]:
print(function6(train, test))